In [ ]:
%%capture
%cd ../../
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import LogNorm
from tqdm import tqdm

from make_clinical_dataset import ROOT_DIR
from make_clinical_dataset.constants import OBS_MAP
from make_clinical_dataset.preprocess.epr.lab import clean_lab_data
from make_clinical_dataset.preprocess.epr.opis import clean_drug_name, filter_treatment_data
from make_clinical_dataset.preprocess.epr.radiology import get_radiology_data
from make_clinical_dataset.util import load_included_drugs, load_included_regimens

pd.set_option('display.max_columns', 110)
pd.set_option('display.max_rows', 110)

In [ ]:
# load external data
included_drugs = load_included_drugs()
included_regimens = load_included_regimens()

mrn_map = pd.read_csv(f'{ROOT_DIR}/data/external/MRN_map.csv')
mrn_map = mrn_map.set_index('RESEARCH_ID')['PATIENT_MRN'].to_dict()

# Lab Data

In [ ]:
biochem = pd.read_parquet(f'{ROOT_DIR}/data/raw/biochemistry.parquet.gzip')
hema = pd.read_parquet(f'{ROOT_DIR}/data/raw/hematology.parquet.gzip')
lab_data = {'Hematology': hema, 'Biochemistry': biochem}

In [ ]:
"""
Check out the lab name groupings
"""
for lab_name, df in lab_data.items():
    assert not (df['component-code-coding-0-display'].notnull() & df['component-code-text'].notnull()).any()
    df['name'] = df['component-code-coding-0-display'].fillna(df['component-code-text'])
    df['group'] = df['name'].map(OBS_MAP[lab_name])
    df['value'] = df['component-valueQuantity-value']

    vals = OBS_MAP[lab_name].values()
    nrows, ncols = df['group'].nunique() // 2 + 1, 2
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8*ncols, 6*nrows))
    axes = axes.flatten()
    i = 0
    for name, group in df.groupby('group'):
        group['value'] = group['value'].clip(upper=group['value'].quantile(0.99))
        palette = sns.color_palette('husl', group['name'].nunique())
        sns.boxplot(data=group, x="name", y="value", ax=axes[i], hue="name")
        axes[i].set(title=f'{name} (N={len(group)})')
        i += 1

    # store the original lab names
    # df['name'].value_counts().to_csv(f'{ROOT_DIR}/data/interim/{lab_name.lower()}_lab_names.csv')

## 'component-code-coding-0-display' vs 'component-code-text'
NOTE: also the timestamp format differs based on which column is used

In [ ]:
for lab_name, df in lab_data.items():
    df = clean_lab_data(df)
    df = df[df['obs_value'].notnull()]
    df['obs_display'] = df['obs_display'].map(OBS_MAP[lab_name])
    df['obs_text'] = df['obs_text'].map(OBS_MAP[lab_name])
    assert df['obs_display'].nunique() == df['obs_text'].nunique()
    lab_data[lab_name] = df

### compare process codes between the two

In [ ]:
df = lab_data['Hematology']
print(df.groupby('obs_display')['proc_name'].value_counts())
print('#'*100)
print(df.groupby('obs_text')['proc_name'].value_counts())

In [ ]:
df = lab_data['Biochemistry']
print(df.groupby('obs_display')['proc_name'].value_counts())
print('#'*100)
print(df.groupby('obs_text')['proc_name'].value_counts())

### compare lab units between the two

In [ ]:
df = lab_data['Hematology']
print(df.groupby('obs_display')['obs_unit'].value_counts())
print('#'*100)
print(df.groupby('obs_text')['obs_unit'].value_counts())

In [ ]:
df = lab_data['Biochemistry']
print(df.groupby('obs_display')['obs_unit'].value_counts())
print('#'*100)
print(df.groupby('obs_text')['obs_unit'].value_counts())

### Observation value distribution

In [ ]:
for lab_name, df in lab_data.items():
    fig, axes = plt.subplots(nrows=df['obs_display'].nunique(), ncols=2, figsize=(20,50))
    clip_thresh = {}
    for col_idx, col in enumerate(['obs_display', 'obs_text']):
        tmp = df[df[col].notnull()]
        for row_idx, (obs_name, obs_value) in enumerate(tmp['obs_value'].groupby(tmp[col])):
            ax = axes[row_idx, col_idx]
            if obs_name not in clip_thresh: 
                clip_thresh[obs_name] = (obs_value.quantile(0.01), obs_value.quantile(0.99))
            obs_value = obs_value.clip(*clip_thresh[obs_name])
            sns.histplot(obs_value, bins=20, kde=True, ax=ax)
            ax.set(title=f'{obs_name} {col}')
    plt.tight_layout()

## Measurement count over time

In [ ]:
lab = pd.read_parquet(f'{ROOT_DIR}/data/interim/lab.parquet.gzip')
lab['obs_year'] = pd.to_datetime(lab['obs_date']).dt.year
cols = lab.columns.drop(['mrn', 'obs_year', 'obs_date'])
counts = lab.groupby('obs_year').apply(lambda g: g[cols].notnull().sum())
counts = counts.reset_index().melt('obs_year', var_name='lab_test', value_name='count')
g = sns.relplot(
    data=counts, x='obs_year', y='count', col='lab_test', col_wrap=3, kind='line', 
    facet_kws={'sharex': False, 'sharey': False}
)

# Symptom Data

In [ ]:
dart = pd.read_parquet(f'{ROOT_DIR}/data/raw/dart.parquet.gzip')
symp = pd.read_parquet(f'{ROOT_DIR}/data/interim/symptom.parquet.gzip')

## Measurement count over time

In [ ]:
symp['survey_year'] = pd.to_datetime(symp['survey_date']).dt.year
cols = symp.columns.drop(['mrn', 'survey_year', 'survey_date'])
counts = symp.groupby('survey_year').apply(lambda g: g[cols].notnull().sum())
counts = counts.reset_index().melt('survey_year', var_name='symptom', value_name='count')
g = sns.relplot(
    data=counts, x='survey_year', y='count', col='symptom', col_wrap=3, kind='line', 
    facet_kws={'sharex': False, 'sharey': False}
)

## Score distribution

In [ ]:
fig, axes = plt.subplots(nrows=int(np.ceil(len(cols)/2)), ncols=2, figsize=(10,25))
axes = axes.flatten()
for idx, col in enumerate(cols): 
    sns.histplot(symp[col], ax=axes[idx], discrete=True)
plt.tight_layout()

## Compare demographic info with cancer registry

In [ ]:
canc_reg = pd.read_parquet(f'{ROOT_DIR}/data/raw/cancer_registry.parquet.gzip')
mrns = list(set(canc_reg['MEDICAL_RECORD_NUMBER']).intersection(dart['MRN']))
canc_reg_sex = canc_reg.groupby('MEDICAL_RECORD_NUMBER')['SEX'].unique()[mrns]
dart_sex = dart.groupby('MRN')['GENDER'].unique()[mrns]
mrns_with_conflict = [mrn for mrn in mrns if any(canc_reg_sex[mrn] != dart_sex[mrn]) or len(dart_sex[mrn]) > 1]
print(f'{len(mrns_with_conflict)} patients with conflicting sex info out of {len(dart_sex)} overlapping patients')

# Treatment Data

In [ ]:
opis = pd.read_parquet(f'{ROOT_DIR}/data/raw/opis.parquet.gzip')
trt = pd.read_parquet(f'{ROOT_DIR}/data/interim/treatment.parquet.gzip')

## Intent

In [ ]:
"""
Usually, after palliative intent treatment, rest of the treatment remains palliative.

Check the number of cases where that is not true.
"""
def check_intent_stays_palliative(df):
    mask = df['intent'] == 'PALLIATIVE'
    if not mask.any():
        return True
    idx = mask[mask].index[0]
    return all(mask.loc[idx:])

mask = trt.groupby('mrn').apply(check_intent_stays_palliative)
mask.value_counts()

In [ ]:
"""Number of treatments over time"""
trt['treatment_year'] = trt['treatment_date'].dt.year
counts = trt.groupby('treatment_year')['intent'].value_counts()
counts = pd.DataFrame(counts).rename(columns={'intent': 'count'}).reset_index()
counts = counts.pivot(index='treatment_year', columns='intent', values='count')
sns.lineplot(data=counts)

## First Treatment Date

In [ ]:
"""
Usually, the first treatment date is monotonically increasing (patients do not continue old treatment after starting new treatment)

Check the number of cases where that is not true.
"""
mask = trt.groupby('mrn').apply(lambda g: all(g['first_treatment_date'] == sorted(g['first_treatment_date'])))
mask.value_counts()

## Height and Weight

In [ ]:
"""Height/weight distribution of patients undertaking treatment""" 
height_and_weight = trt.groupby('mrn').agg({'height': 'mean', 'weight': 'mean'})
sns.displot(data=height_and_weight, x='height', y='weight')

## Drugs

In [ ]:
"""Number of drugs administered over time"""
trt['treatment_year'] = trt['treatment_date'].dt.year
cols = trt.columns
drug_cols = cols[cols.str.endswith('given_dose')]
counts = trt.groupby('treatment_year').apply(lambda g: (g[drug_cols] != 0).sum(), include_groups=False)
counts = counts.rename(columns={col: col.replace('drug_', '').replace('_given_dose', '') for col in drug_cols})
counts = counts.reset_index().melt('treatment_year', var_name='drug', value_name='count')
g = sns.relplot(
    data=counts, x='treatment_year', y='count', col='drug', col_wrap=3, kind='line', 
    facet_kws={'sharex': False, 'sharey': False}
)

In [ ]:
"""
Check the number of cases where the dose given differed from dose ordered
"""
mask = (opis['Dose_Given'] == opis['Dose_Ord'])
mask.value_counts()

In [ ]:
"""
Check out the drug groupings
"""
import logging; logging.disable(logging.CRITICAL) # temporarily disable logging

df = filter_treatment_data(opis, included_drugs, included_regimens)
nrows, ncols = df['drug_name'].nunique() // 2 + 1, 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8*ncols, 6*nrows))
axes = axes.flatten()
for i, (drug_name, group) in enumerate(df.groupby('drug_name')):
    # if group['orig_drug_name'].nunique() < 2: continue
    palette = sns.color_palette('husl', group['orig_drug_name'].nunique())
    sns.boxplot(data=group, x="orig_drug_name", y="given_dose", ax=axes[i], hue="orig_drug_name")
    axes[i].set(title=f'{drug_name} (N={len(group)})')

In [ ]:
# how often regimen_dose is NaN, which leads to %_ideal_dose to be NaN too
def helper(g):
    N = g['regimen_dose'].isna().sum()
    return f"{N} ({N/len(g)*100:.2f}%)"
df.groupby('drug_name').apply(helper, include_groups=False)

## Days Since Prev Session

In [ ]:
# check the number of cases for each days since prev session
pd.concat([
    trt['days_since_last_treatment'].value_counts(),
    trt.groupby('days_since_last_treatment').apply(lambda g: g['mrn'].nunique())
], axis=1, keys=['Sessions', 'Patients']).sort_index().head(n=100)

# Emerg Data

In [ ]:
er_visit = pd.read_parquet(f'{ROOT_DIR}/data/raw/ER.parquet.gzip')
erv = pd.read_parquet(f'{ROOT_DIR}/data/interim/emergency_room_visit.parquet.gzip')

## ER Reason

In [ ]:
erv['CEDIS_complaint'].value_counts().head(100)

In [ ]:
# how often do CEDIS_complaint and chief_complaint deviate
mask = erv['CEDIS_complaint'].str.lower() == erv['chief_complaint'].str.lower()
mask.value_counts()

# Death Data

In [ ]:
cancer_registry_ddates = pd.read_parquet('./data/raw/cancer_registry.parquet.gzip')
epr_death_ddates = pd.read_parquet('./data/raw/death_dates.parquet.gzip')
cancer_registry_ddates.rename(columns={'MEDICAL_RECORD_NUMBER': 'mrn', 'DATE_OF_DEATH': 'date_of_death'}, inplace=True)
epr_death_ddates.rename(columns={'MEDICAL_RECORD_NUMBER': 'mrn', 'DATE_OF_DEATH': 'date_of_death'}, inplace=True)
epr_death_ddates['date_of_death'] = pd.to_datetime(epr_death_ddates['date_of_death'], format='%d%b%Y:%H:%M:%S')
assert epr_death_ddates['date_of_death'].notnull().all() and not epr_death_ddates['mrn'].duplicated().any()
df = pd.merge(cancer_registry_ddates, epr_death_ddates, on ='mrn', how='left', suffixes=('_cr', '_epr'))

In [ ]:
mask1 = df[['date_of_death_cr', 'date_of_death_epr']].notnull().all(axis=1)
mask2 = df['date_of_death_cr'].dt.date != df['date_of_death_epr']
tmp = df[mask1 & mask2]
diff = tmp['date_of_death_cr'] - tmp['date_of_death_epr']
mask = diff.dt.days.abs() > 3
conflicting_death_dates = tmp[mask].groupby('mrn')[['date_of_death_cr', 'date_of_death_epr']].first()
print(f'There are {len(conflicting_death_dates)} patients with conflicting death dates greater than 3 days')
mask = df['date_of_death_epr'].notnull() & df['date_of_death_cr'].isnull()
print(f'There are {sum(mask)} patients with death dates in the EPR death dataset but not in the Cancer Registry dataset')
mask = df['date_of_death_cr'].notnull() & df['date_of_death_epr'].isnull()
print(f'There are {sum(mask)} patients with death dates in the Cancer Registry dataset but not in EPR death dataset')

# Demographic / Cancer Data

In [ ]:
dmg = pd.read_parquet(f'{ROOT_DIR}/data/interim/demographic.parquet.gzip')
canc_cols = [col for col in dmg.columns if col.startswith('cancer_site')]

In [ ]:
# number of patients diagnosed with multiple cancers
mask = dmg[canc_cols].notnull().sum(axis=1) > 1
mask.value_counts()

In [ ]:
# number of patients diagnosed with two different cancers on the same day
mask = dmg[cols].nunique(axis=1) != dmg[cols].notnull().sum(axis=1)
mask.value_counts()

# Radiology Data

## Duplicate reports

In [ ]:
reports = get_radiology_data(mrn_map, data_dir=f'{ROOT_DIR}/data/raw', drop_duplicates=False)

In [ ]:
mask = reports.duplicated(subset=['mrn', 'raw_text', 'proc_name'], keep=False)
print(f'{sum(mask)} duplicated reports with same proc_name found out of {len(mask)} reports')
mask = reports.duplicated(subset=['mrn', 'raw_text'], keep=False)
print(f'{sum(mask)} duplicated reports found out of {len(mask)} reports')
duplicated_reports = reports[mask].sort_values(by='epr_datetime')

In [ ]:
time_diffs = []
for (mrn, text), g in tqdm(duplicated_reports.groupby(['mrn', 'raw_text'])):
    time_diff = g['epr_datetime'] - g['epr_datetime'].iloc[0]
    time_diff = time_diff[time_diff != pd.Timedelta(seconds=0)]
    time_diffs.append(time_diff[1:])

In [ ]:
time_diffs = pd.concat(time_diffs)
bins = [
    pd.Timedelta(minutes=0),
    pd.Timedelta(hours=1),
    pd.Timedelta(hours=2),
    pd.Timedelta(hours=5),
    pd.Timedelta(hours=24),
    pd.Timedelta(days=2),
    pd.Timedelta(days=5),
    pd.Timedelta(days=5000),
]
labels = [
    '0-1hour',
    '1-2hours', 
    '2-5hours', 
    '5-24hours', 
    '1-2days', 
    '2-5days', 
    '5days+'
]
time_diffs = pd.cut(time_diffs, bins=bins, labels=labels)

In [ ]:
ax = sns.histplot(data=time_diffs)
ax.set(title='Entry timestamp difference between first report and duplicated report')
ax.tick_params(axis='x', rotation=45)

In [ ]:
paired_proc_names = duplicated_reports['proc_name'].unique()
paired_proc_names = pd.DataFrame(1, columns=paired_proc_names, index=paired_proc_names).to_dict()
for (mrn, text), g in tqdm(duplicated_reports.groupby(['mrn', 'raw_text'])):
    proc_names = g['proc_name'].unique()
    for i, proc_name1 in enumerate(proc_names):
        for proc_name2 in proc_names[i+1:]:
            paired_proc_names[proc_name1][proc_name2] += 1
paired_proc_names = pd.DataFrame(paired_proc_names)

drop = [name for name in paired_proc_names if all(paired_proc_names[name] == 0) & all(paired_proc_names.loc[name] == 0)]
paired_proc_names = paired_proc_names.drop(index=drop, columns=drop)

# drop = [name for name in paired_proc_names if 'CT' not in name]
# paired_proc_names = paired_proc_names.drop(index=drop, columns=drop)

keep = paired_proc_names.sum().sort_values(ascending=False).index[:20]
paired_proc_names = paired_proc_names.loc[keep, keep]

fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(paired_proc_names, square=True, xticklabels=True, yticklabels=True, ax=ax, norm=LogNorm())

# No Overlaps

In [ ]:
def get_overlap_numbers(a, b, a_name, b_name):
    set_a, set_b = set(a), set(b)
    return pd.DataFrame(
        data=[len(set_a - set_b), len(set_a & set_b), len(set_b - set_a)],
        index=[f'Only {a_name}', 'In both', f'Only {b_name}']
    ).T

In [ ]:
biochem = pd.read_parquet(f'{ROOT_DIR}/data/raw/biochemistry.parquet.gzip')
hema = pd.read_parquet(f'{ROOT_DIR}/data/raw/hematology.parquet.gzip')
mrn_map = pd.read_csv(f'{ROOT_DIR}/data/external/MRN_map.csv')

In [ ]:
# double checking the mrn mapping is unique
assert mrn_map['RESEARCH_ID'].nunique() == mrn_map['PATIENT_MRN'].nunique() == len(mrn_map)
print(f'{mrn_map["RESEARCH_ID"].nunique()} patients in the MRN map')
# all patients in biochemistry and hematology have a MRN (overlaps with MRN map)
assert biochem['patientid'].isin(mrn_map['RESEARCH_ID']).all()
assert hema['patientid'].isin(mrn_map['RESEARCH_ID']).all()
get_overlap_numbers(biochem['patientid'], hema['patientid'], 'biochem', 'hema')

In [ ]:
trt = pd.read_parquet(f'{ROOT_DIR}/data/interim/treatment.parquet.gzip')
dmg = pd.read_parquet(f'{ROOT_DIR}/data/interim/demographic.parquet.gzip')
get_overlap_numbers(trt['mrn'], dmg['mrn'], 'treatment', 'demographic')

In [ ]:
trt = pd.read_parquet(f'{ROOT_DIR}/data/interim/treatment.parquet.gzip')
lab = pd.read_parquet(f'{ROOT_DIR}/data/interim/lab.parquet.gzip')
get_overlap_numbers(trt['mrn'], lab['mrn'], 'treatment', 'laboratory')

In [ ]:
df = trt[['treatment_date']].copy()
df['overlap'] = trt['mrn'].isin(lab['mrn'])
sns.histplot(data=df, x='treatment_date', hue='overlap')

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))

df = trt[['regimen', 'mrn']].copy()
df['overlap'] = df['mrn'].isin(lab['mrn'])
sns.histplot(data=df, y='regimen', hue='overlap', ax=ax1)
ax1.set(xlabel='Count (Sessions)')
ax1.yaxis.set_tick_params(labelsize=6)

df = df.drop_duplicates(subset=['regimen', 'mrn'])
sns.histplot(data=df, y='regimen', hue='overlap', ax=ax2)
ax2.set(xlabel='Count (Patients)', ylabel=None)
ax2.yaxis.set_tick_params(labelleft=False)

plt.tight_layout()

In [ ]:
df = pd.read_parquet(f'{ROOT_DIR}/data/processed/treatment_centered_clinical_dataset.parquet.gzip')
lab = pd.read_parquet(f'{ROOT_DIR}/data/interim/lab.parquet.gzip')
get_overlap_numbers(df['mrn'], lab['mrn'], 'final dataset', 'laboratory')

In [ ]:
from ml_common.constants import CANCER_CODE_MAP
cancer_site_cols = df.columns[df.columns.str.contains("cancer_site")]
df["cancer_site"] = df[cancer_site_cols].apply(lambda mask: ", ".join(cancer_site_cols[mask].str.removeprefix("cancer_site_")), axis=1)
df["Cancer Site"] = df["cancer_site"].replace(CANCER_CODE_MAP)
top_20_cancer_sites = df['Cancer Site'].value_counts().index[:20]
df = df[df['Cancer Site'].isin(top_20_cancer_sites)]
df['overlap'] = df['mrn'].isin(lab['mrn'])
sns.histplot(data=df, y='Cancer Site', hue='overlap', stat='proportion')

In [ ]:
trt = pd.read_parquet(f'{ROOT_DIR}/data/interim/treatment.parquet.gzip')
symp = pd.read_parquet(f'{ROOT_DIR}/data/interim/symptom.parquet.gzip')
get_overlap_numbers(trt['mrn'], symp['mrn'], 'treatment', 'symptom')

In [ ]:
df = trt[['treatment_date']].copy()
df['overlap'] = trt['mrn'].isin(symp['mrn'])
sns.histplot(data=df, x='treatment_date', hue='overlap')